In [1]:
import torch

print("✅ Torch:", torch.__version__)
print("✅ CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ GPU не найдена — обучение пойдёт на CPU")


✅ Torch: 2.9.1+cu130
✅ CUDA available: True
✅ GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
from tqdm import tqdm
import shutil
import random

# Пути к исходным данным
train_img_dir = r"D:\7 sem\\mashine learn\datasets\DroneTrainDataSet\Drone_TrainSet"
train_xml_dir = r"D:\7 sem\\mashine learn\datasets\DroneTrainDataSet\Drone_TrainSet_XMLs"
val_img_dir = r"D:\7 sem\\mashine learn\datasets\DroneTestDataset\Drone_TestSet"
val_xml_dir = r"D:\7 sem\\mashine learn\datasets\DroneTestDataset\Drone_TestSet_XMLs"

# Новая структура
base_out = "datasets/drone_custom"
os.makedirs(f"{base_out}/images/train", exist_ok=True)
os.makedirs(f"{base_out}/images/val", exist_ok=True)
os.makedirs(f"{base_out}/labels/train", exist_ok=True)
os.makedirs(f"{base_out}/labels/val", exist_ok=True)

# === Настройки ===
TRAIN_LIMIT = 10_000   # максимум файлов для train
VAL_LIMIT = None        # можно поставить лимит и для валидации

def convert_voc_to_yolo(img_path, xml_path, out_label_path):
    """Преобразует 1 XML в YOLO txt"""
    tree = ET.parse(xml_path)
    root = tree.getroot()

    img = Image.open(img_path)
    w, h = img.size

    lines = []
    for obj in root.findall("object"):
        cls = obj.find("name").text.strip().lower()
        if cls not in ["drone", "uav"]:
            continue

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)

        x_center = ((xmin + xmax) / 2) / w
        y_center = ((ymin + ymax) / 2) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h

        # Один класс: drone -> 0
        lines.append(f"0 {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}")

    if lines:
        with open(out_label_path, "w") as f:
            f.write("\n".join(lines))

def process_split(img_dir, xml_dir, split, limit=None):
    out_img_dir = f"{base_out}/images/{split}"
    out_label_dir = f"{base_out}/labels/{split}"

    # Список изображений
    all_imgs = [f for f in os.listdir(img_dir)
                if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    all_imgs.sort()

    # Ограничиваем количество
    if limit and len(all_imgs) > limit:
        all_imgs = random.sample(all_imgs, limit)

    for file in tqdm(all_imgs, desc=f"Processing {split}", unit="img"):
        img_path = os.path.join(img_dir, file)
        xml_path = os.path.join(xml_dir, os.path.splitext(file)[0] + ".xml")

        if not os.path.exists(xml_path):
            continue

        # Копируем изображение
        shutil.copy2(img_path, os.path.join(out_img_dir, file))

        # Конвертируем аннотацию
        out_label_path = os.path.join(out_label_dir, os.path.splitext(file)[0] + ".txt")
        convert_voc_to_yolo(img_path, xml_path, out_label_path)

# Обработка train (с лимитом 10к)
process_split(train_img_dir, train_xml_dir, "train", limit=TRAIN_LIMIT)

# Обработка val (без ограничения)
process_split(val_img_dir, val_xml_dir, "val", limit=VAL_LIMIT)

print(f"✅ Конвертация завершена.\nФайлы сохранены в: {base_out}")


Processing val: 100%|██████████| 5375/5375 [02:48<00:00, 31.85img/s]   

✅ Конвертация завершена.
Файлы сохранены в: datasets/drone_custom


In [2]:
from ultralytics import YOLO
import torch

device = 0 if torch.cuda.is_available() else 'cpu'
print("Используется устройство:", device)

model = YOLO("runs/detect/drone_uav_experiment_2/weights/best.pt") 

model.train(
    data="drone_custom.yaml",
    epochs=21,
    imgsz=768,
    batch=32,
    device=device,
    workers=0,
    name="drone_uav_experiment_3"
)


Используется устройство: 0
New https://pypi.org/project/ultralytics/8.3.228 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.227  Python-3.13.8 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=drone_custom.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=21, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/drone_uav_experiment_2/weights/best.pt, momentum=0.93

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001918D0CAB10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:
from ultralytics import YOLO
import torch
model = YOLO("runs/detect/drone_uav_experiment/weights/best.pt") 
metrics = model.val(data="drone_custom.yaml", batch=8, imgsz=640)
print(metrics)

: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

results = pd.read_csv('runs/detect/drone_uav_experiment_3/results.csv')

# График Recall
plt.plot(results['epoch'], results['metrics/recall(B)'])
plt.title('Recall during training')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.show()

# Последнее значение Recall
final_recall = max(results['metrics/recall(B)'])
print(f"Final Recall: {final_recall:.3f}")

In [6]:
import cv2
from ultralytics import YOLO
import torch

device = 0 if torch.cuda.is_available() else 'cpu'
model = YOLO("runs/detect/drone_uav_experiment_3/weights/best.pt")

input_video = "20251112-1020-20.9685021.mp4"
output_video = "mass2.mp4"

cap = cv2.VideoCapture(input_video)
fps = cap.get(cv2.CAP_PROP_FPS) or 25
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w,h))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model.predict(frame, conf=0.35, device=device, verbose=False)
    annotated = results[0].plot()
    out.write(annotated)

cap.release()
out.release()
print("🎯 Видео с детекцией сохранено:", output_video)


🎯 Видео с детекцией сохранено: mass2.mp4


In [ ]:
import cv2
import os

# Путь к видео
video_path = "video_2025-11-17_11-21-32.mp4"

# Папка для сохранения кадров
output_folder = "frtest"

# Создаём папку, если её нет
os.makedirs(output_folder, exist_ok=True)

# Загружаем видео
cap = cv2.VideoCapture(video_path)

frame_id = 0

while True:
    ret, frame = cap.read()
    
    # Если кадры кончились — выходим
    if not ret:
        break
    
    # Имя файла кадра
    frame_path = os.path.join(output_folder, f"frame_{frame_id:05d}.jpg")
    
    # Сохраняем кадр как изображение
    cv2.imwrite(frame_path, frame)
    
    frame_id += 1

cap.release()
print("Готово! Кадры сохранены в:", output_folder)


Готово! Кадры сохранены в: frtest
